### LangChain Output Parsers

In [1]:
import os

from dotenv import load_dotenv
from langchain.chains import LLMChain
from langchain_openai import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

In [2]:
load_dotenv(override=True)

azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_deployment = os.getenv("AZURE_DEPLOYMENT_NAME")
azure_api_version = os.getenv("AZURE_API_VERSION")

if not azure_endpoint or not azure_api_key or not azure_deployment or not azure_api_version:
    raise ValueError("Azure OpenAI environment variables are not set.")

temperature = 0.7
max_tokens = 1500

llm = AzureChatOpenAI(
    azure_endpoint=azure_endpoint,
    api_key=azure_api_key,
    azure_deployment=azure_deployment,
    api_version=azure_api_version,
    temperature=temperature,
    max_tokens=max_tokens,
    model="gpt-4o"
)

In [3]:
response_schema = [
    ResponseSchema(
        name="bad_string",
        description="This is poorly formatted user response"
    ),
    ResponseSchema(
        name="good_string",
        description="This is well and properly formatted user response"
    ),
]

output_parser = StructuredOutputParser.from_response_schemas(response_schema)

format_instructions = output_parser.get_format_instructions()

print("Format Instructions:")
print(format_instructions)

Format Instructions:
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"bad_string": string  // This is poorly formatted user response
	"good_string": string  // This is well and properly formatted user response
}
```


In [4]:
template = """
    You shall be given a poorly formatted string from a user.
    Reformat it and make sure all the words are spelled correctly.
    
    Can you please provide both the original poorly formatted string and the corrected well formatted, business professional
    string in your response?
    
    {format_instructions}
    
    User Input: {user_input}
    
    YOUR RESPONSE:
"""

prompt = PromptTemplate(
    input_variables=["user_input"],
    template=template,
    partial_variables={"format_instructions": format_instructions}
)

prompt_value = prompt.format(
    user_input="""
        Welcome to Londan! I hope u hv a great time her. 
        The wether is nice and the foody is delicious. 
        Don't forgat to visit the Big Ban and the Towar of Londan. Enjoy your stey!
        
        Here is the conversation I (My name is Alex) had with my frend:
        
            Alex: Hey, I'm thinking about watching a movie this weekend. Do you have any recommendations?
            Ben: Oh, what kind of movie are you in the mood for?
            Alex: I'm pretty open, but I like action or comedies. What's playing?
            Ben: Well, there's that new sci-fi action movie, "Galactic Pursuit." It's getting good reviews.
            Alex: Galactic Pursuit? What's the premise?
            Ben: It's about a team of space explorers who have to team up with an unlikely ally to stop an alien invasion.
            Alex: That sounds great, thanks for the recommendation!
            Ben: No problem! Let me know what you think if you end up seeing it.
            Alex: Will do. See you later! 
    """
)

In [5]:
llm_output = llm.invoke(
    prompt_value,
)

print("LLM Output:")
print(llm_output)

LLM Output:
content='```json\n{\n\t"bad_string": "Welcome to Londan! I hope u hv a great time her. The wether is nice and the foody is delicious. Don\'t forgat to visit the Big Ban and the Towar of Londan. Enjoy your stey!\\n\\nHere is the conversation I (My name is Alex) had with my frend:\\n\\n    Alex: Hey, I\'m thinking about watching a movie this weekend. Do you have any recommendations?\\n    Ben: Oh, what kind of movie are you in the mood for?\\n    Alex: I\'m pretty open, but I like action or comedies. What\'s playing?\\n    Ben: Well, there\'s that new sci-fi action movie, \\"Galactic Pursuit.\\" It\'s getting good reviews.\\n    Alex: Galactic Pursuit? What\'s the premise?\\n    Ben: It\'s about a team of space explorers who have to team up with an unlikely ally to stop an alien invasion.\\n    Alex: That sounds great, thanks for the recommendation!\\n    Ben: No problem! Let me know what you think if you end up seeing it.\\n    Alex: Will do. See you later!",\n\t"good_string

In [6]:
result = output_parser.parse(llm_output.content)
print("Parsed Result:")
print(result)
print("Bad String:", result["bad_string"])
print("Good String:", result["good_string"])

Parsed Result:
{'bad_string': 'Welcome to Londan! I hope u hv a great time her. The wether is nice and the foody is delicious. Don\'t forgat to visit the Big Ban and the Towar of Londan. Enjoy your stey!\n\nHere is the conversation I (My name is Alex) had with my frend:\n\n    Alex: Hey, I\'m thinking about watching a movie this weekend. Do you have any recommendations?\n    Ben: Oh, what kind of movie are you in the mood for?\n    Alex: I\'m pretty open, but I like action or comedies. What\'s playing?\n    Ben: Well, there\'s that new sci-fi action movie, "Galactic Pursuit." It\'s getting good reviews.\n    Alex: Galactic Pursuit? What\'s the premise?\n    Ben: It\'s about a team of space explorers who have to team up with an unlikely ally to stop an alien invasion.\n    Alex: That sounds great, thanks for the recommendation!\n    Ben: No problem! Let me know what you think if you end up seeing it.\n    Alex: Will do. See you later!', 'good_string': 'Welcome to London! I hope you have